In [ ]:
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
img_dir = '/content/drive/My Drive/CelebA/Img/img_align_celeba_32500'
# Load labels.csv
labels_df = pd.read_csv('/content/drive/My Drive/CelebA/Anno/labels_even.csv', header=0) 

# Load list_bbox_celeba.txt
bbox_df = pd.read_csv('/content/drive/My Drive/CelebA/Anno/list_bbox_celeba.txt', delim_whitespace=True, header=0, dtype=object) 

# rename column 'image_id' of bbox_df to 'Filename'
bbox_df.rename(columns={'image_id': 'Filename'}, inplace=True)

# Convert 'x_1', 'y_1', 'width' and 'height' columns of bbox_df to numeric
bbox_df['x_1'] = pd.to_numeric(bbox_df['x_1'])
bbox_df['y_1'] = pd.to_numeric(bbox_df['y_1'])
bbox_df['width'] = pd.to_numeric(bbox_df['width'])
bbox_df['height'] = pd.to_numeric(bbox_df['height'])

# Merge labels_df and bbox_df
merged_df = pd.merge(labels_df, bbox_df, on='Filename')

In [ ]:
merged_df['x_normalized'] = merged_df['x_1'] / merged_df['width']
merged_df['y_normalized'] = merged_df['y_1'] / merged_df['height']
merged_df['width_normalized'] = merged_df['width'] / merged_df['width']
merged_df['height_normalized'] = merged_df['height'] / merged_df['height']

# remove columns 'x_1', 'y_1', 'width' and 'height'
merged_df.drop(['x_1', 'y_1', 'width', 'height'], axis=1, inplace=True)

In [ ]:
class CustomImageDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, dataframe, batch_size, image_size, class_names):
        self.dataframe = dataframe
        self.batch_size = batch_size
        self.image_size = image_size
        self.class_names = class_names
        self.num_classes = len(class_names)
        self.indexes = np.arange(len(dataframe))

    def __len__(self):
        return int(np.ceil(len(self.dataframe) / self.batch_size))

    def __getitem__(self, index):
        batch_indexes = self.indexes[index * self.batch_size: (index + 1) * self.batch_size]
        batch_data = self.dataframe.iloc[batch_indexes]

        batch_images = []
        batch_classes = []

        for _, row in batch_data.iterrows():
            image = cv2.imread(img_dir + '/' + row['Filename'])
            if image is None:
                print(f"Error loading image: {row['Filename']}")
                continue
            
            # Resize image to desired size
            #image = cv2.resize(image, self.image_size)
            # Normalize image
            image = image.astype('float32') / 255.0
             
            class_vector = [row[class_name] for class_name in self.class_names]  # Get class labels
            one_hot_class = tf.keras.utils.to_categorical(class_vector, num_classes=self.num_classes)
            batch_images.append(image)
            batch_classes.append(class_vector)

        # Convert lists to NumPy arrays
        batch_of_images = np.array(batch_images)
        batch_of_classes = np.array(batch_classes)

        return (
            batch_of_images,
            batch_of_classes
        )

# List of class names
class_names = labels_df.columns[1:].tolist()
print(class_names)

# Image size
image_size = (218,178)

# Batch size
batch_size = 32

# Create data generator
data_generator = CustomImageDataGenerator(merged_df, batch_size, image_size, class_names)


In [ ]:
from google.colab.patches import cv2_imshow

image = cv2.imread(img_dir + '/' + '000001.jpg')
image = image.astype('float32') / 255.0
# view the shape of the image
print(image.shape)
# view the image
cv2_imshow(image)

for epoch in range(epochs):
    for batch_images, batch_data in data_generator:
        # Train your model using batch_images and batch_data
        model.train_on_batch(batch_images, [batch_data['class_names'], batch_data['bboxes']])


In [ ]:
def preprocess_images_and_bboxes(images, bboxes, target_size):
    cropped_images = []
    for image, bbox in zip(images, bboxes):
        x, y, width, height = bbox
        x_min = int(x * image.shape[1])
        y_min = int(y * image.shape[0])
        x_max = int((x + width) * image.shape[1])
        y_max = int((y + height) * image.shape[0])
        
        cropped_image = image[y_min:y_max, x_min:x_max]
        resized_image = cv2.resize(cropped_image, target_size)
        
        cropped_images.append(resized_image)
    
    return np.array(cropped_images)


In [ ]:
import math
import matplotlib.pyplot as plt

def debug_data_generator(generator):
    # Generate one batch of data
    batch_of_images, batch_of_classes = generator[0]
    print("Batch Images Shape:", batch_of_images.shape)
    print("Batch Classes Shape:", batch_of_classes.shape)
    print("Batch Images Type:", type(batch_of_images))
    print("Batch Classes Type:", type(batch_of_classes))
    
    # Calculate the number of rows and columns for the grid layout
    num_images = batch_of_images.shape[0]
    num_cols = 5  # You can adjust the number of columns in the grid
    num_rows = math.ceil(num_images / num_cols)
    
    # Set up the grid layout
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 15))
    
    # Iterate over images and labels, and display them in the grid
    for i, ax in enumerate(axes.flat):
        if i < num_images:
            ax.imshow(batch_of_images[i],cmap='Accent')
            ax.set_title("Class Labels: " + str(batch_of_classes[i]))
            ax.axis('off')
        else:
            ax.axis('off')  # Turn off empty subplots if any
        
    plt.tight_layout()
    plt.show()

# Assuming you have already created the data generator
data_generator = CustomImageDataGenerator(merged_df, batch_size, image_size, class_names)

# Call the debug function to inspect a batch
debug_data_generator(data_generator)


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import Precision, Recall, AUC

# Load pre-trained VGG16 model without top layers
base_model = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(218, 178, 3)
)

num_classes = len(class_names)

# Define input layer (adjust the shape based on your input images)
input_tensor = Input(shape=(218, 178, 3), name='batch_output')

# Pass input through VGG16 base model
x = base_model(input_tensor)

# Add GlobalAveragePooling2D layer to reduce spatial dimensions
x = GlobalAveragePooling2D()(x)

# Add output layer for class predictions
class_output = Dense(num_classes, activation='softmax', name='class_output')(x)

# Create the model with a specific name
model = Model(inputs=input_tensor, outputs=class_output, name='VGG16_transfer')

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy', 'precision', 'recall']
)

# Print model summary
model.summary()

In [ ]:
import time

# Number of epochs for training
epochs = 10

# Initialize timer for overall training time
start_time = time.time()

# Training loop
for epoch in range(epochs):
    # Record start time for the epoch
    epoch_start_time = time.time()
    
    # Initialize timer for epoch time
    batch_start_time = time.time()
    
    # Iterate through batches in the generator
    for batch_images, batch_classes in data_generator:
        # Record start time for the batch
        batch_start_time = time.time()
        
        # Train your model using batch_images and batch_data
        losses = model.train_on_batch(x=batch_images, y=batch_classes,reset_metrics=False)
        
        # Record end time for the batch
        batch_end_time = time.time()
        batch_time = batch_end_time - batch_start_time  # Calculate time taken for the batch
        
        print(f'Epoch {epoch + 1}, Batch Time: {batch_time:.2f}s, Losses: {losses}')
    
    # Record end time for the epoch
    epoch_end_time = time.time()
    epoch_time = epoch_end_time - epoch_start_time  # Calculate time taken for the epoch
    
    print(f'Epoch {epoch + 1} took {epoch_time:.2f} seconds.')
    
# Calculate total training time
end_time = time.time()
total_time = end_time - start_time
print(f'Training completed in {total_time:.2f} seconds.')
